In [1]:
# syft absolute
import syft as sy

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft


### Notes:
* DatasetEvent:
  * A dataset does not have a mock and a real - that is an Asset
  * Assets, however, should not really have an event, because they do not exists in the database as their own entities. They are only subcontainers for two (or one) ActionObjects, by storing the objects ids.
  * Therefore we want an Event for changing an ActionObject
  * This is hard because AOs are connected between low-high side nodes through the dataset/asset they are contained within (which are connected through name or DOs own imagination) 
  * Datasets never change (we dont have a method for that)
* In which ways can we affect ActionObjects?
  * when we create an AO -> CreateObjectEvent (or CreateDatasetEvent?)
  * when we set a new value to an already existing AO -> UpdateDatasetEvent?
  * when we change its permissions? -> ???Event
  * 

### Node creation

In [2]:
low_node = sy.orchestra.launch(
    name="test_low_side",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)
high_node = sy.orchestra.launch(
    name="test_high_side",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/61bc8c1e1c4f44fd84c390416f16fa56.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=85966db2e5014c1685fbbedd01905544 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/11d0605c753f46d7bdcc49e66da97386.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=6d079d69a6ee4ab5a5354ab5d7e2dfde in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


### DS registration

In [3]:
do_low_client = low_node.login(email="info@openmined.org", password="changethis")
do_high_client = high_node.login(email="info@openmined.org", password="changethis")

Logged into <test_low_side: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_high_side: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
# syft absolute
from syft.service.user.user import UserCreate

do_low_client.users.create(
    UserCreate(email="newuser@openmined.org", name="John Doe", password="pw")
)

```python
class UserView:
  id: str = b2d93aced4674f868559c507ea1db85d
  name: str = "John Doe"
  email: str = "newuser@openmined.org"
  institution: str = None
  website: str = None
  role: str = ServiceRole.DATA_SCIENTIST

```

In [5]:
ds_client = low_node.login(email="newuser@openmined.org", password="pw")

Logged into <test_low_side: Low side Domain> as <newuser@openmined.org>


### Dataset upload

In [6]:
# third party
import numpy as np

mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=private_low,
            shape=private_low.shape,
            mock_is_real=True,
        )
    ],
)

do_low_client.upload_dataset(dataset_low)

  0%|                                                                                                                 | 0/1 [00:00<?, ?it/s]

Uploading: numpy-data


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.85it/s]


SyftSuccess: Dataset uploaded to 'test_low_side'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [7]:
# third party
import numpy as np

mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

do_high_client.upload_dataset(dataset_high)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.02it/s]


Uploading: numpy-data


SyftSuccess: Dataset uploaded to 'test_high_side'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [9]:
# high_node.python_node.get_service("EventService").stash.partition.unique_cks

In [10]:
# do_high_client.api.services.event.get_all()[0]

In [11]:
# do_low_client.api.services.event.get_all()[0]

### DS creates and tests UserCode 

In [ ]:
data_low = ds_client.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    return data.mean()


compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///tmp/9f83a76909f5422697b2837008c2b658.sqlite

Consumer service Name:  None
Creating Default Worker Image
Building Default Worker Image
Creating default Worker Pool
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_427: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_427
SyftInfo: Landing the ephmeral node...


```python
Pointer
```
2.0

In [ ]:
ds_client.code.request_code_execution(compute_mean)

syft.service.request.request.Request

In [ ]:
res = ds_client.code.compute_mean(data=data_low)
display(res)

SyftError: UserCodeStatus.DENIED: Function has no output policy

### DO approves on low side machine and moves on high side

In [ ]:
request_low = do_low_client.requests[-1]

request_low.code

```python
class UserCode
    id: UID = 7a8b19e3d6664c40a9be9f1d16ebcd66
    service_func_name: str = compute_mean
    shareholders: list = ['test_low_side']
    status: list = ['Node: test_low_side, Status: pending']
    
    code:

@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    return data.mean()

```

In [ ]:
do_high_client.code.sync_code_from_request(request_low)

SyftSuccess: User Code Submitted

### DO syncs result

In [ ]:
do_high_client._fetch_api(do_high_client.credentials)
data_high = do_high_client.datasets[0].assets[0]

job_high = do_high_client.code.compute_mean(data=data_high, blocking=False)
display(job_high)

```python
class Job:
    id: UID = e87138f7b68b45179244a0db90b8160f
    status: JobStatus.CREATED
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [ ]:
# View and submit job info from high-side

# syft absolute

job_info = job_high.info(public_metadata=True)
display(job_info)

syft.service.job.job_stash.JobInfo

In [ ]:
request_low.sync_job(job_info)

SyftSuccess: Great Success!

In [ ]:
result_high = job_high.wait().get()
print(result_high)

SyftInfo: Node Landed!
17.0


In [ ]:
job_info = job_high.info(result=True)
job_info

syft.service.job.job_stash.JobInfo

In [ ]:
request_low.accept_by_depositing_result(job_info)

Approving request for domain test_low_side
Approving request for domain test_low_side


SyftSuccess: Request submitted for updating result.

### DO gets their results

In [ ]:
ds_client.code.compute_mean(data=data_low).get()

17.0